Imports

In [15]:
from PIL import Image
import numpy as np
import math
import cv2
from pylfsr import LFSR

## Helper Functions

- Any type to Binary Converter


In [16]:
def convertToBinary(data):
    if isinstance(data, str):
        return ''.join([ format(ord(i), "08b") for i in data ])
    elif isinstance(data, bytes) or isinstance(data, np.ndarray):
        return [ format(i, "08b") for i in data ]
    elif isinstance(data, int) or isinstance(data, np.uint8):
        return format(data, "08b")
    else:
        raise TypeError("Type not supported.")

In [17]:
print(list(convertToBinary(256)))

['1', '0', '0', '0', '0', '0', '0', '0', '0']


- Binary to Decimal Converter

In [18]:
def binaryToDecimal(binary):
    decimal = 0
    for bit in binary:
        decimal = decimal*2 + int(bit)
    return decimal

- Convert image to matrix

In [19]:
def getImageMatrix(imageName):

    imageHandler = Image.open(imageName)
    pixels = imageHandler.load()
    color = 1
    if type(pixels[0,0]) == int:
        color = 0
    image_size = imageHandler.size
    
    rows = int(image_size[0])
    cols = int(image_size[1])
    image_matrix = []
    for row in range(rows):
        current_row = []
        for col in range(cols):
            current_row.append(pixels[row,col])
        image_matrix.append(current_row)
    
    return image_matrix, image_size, color


- Extract Frames From Video

In [36]:
def FrameCapture(path): 
  
    vidObj = cv2.VideoCapture(path) 
    count = 0
  
    success = 1
  
    while count < 100: 
        success, image = vidObj.read()
        if not success:
            print("Video End")
            return  
        count += 1
    
    cv2.imwrite("images/frame1.jpg", image) 

    while count < 200: 
        success, image = vidObj.read() 
        if not success:
            print("Video End")
            return 
        count += 1

    cv2.imwrite("images/frame2.jpg", image) 

    while count < 300: 
        success, image = vidObj.read() 
        if not success:
            print("Video End")
            return 
        count += 1

    cv2.imwrite("images/frame3.jpg", image) 

    while count < 400: 
        success, image = vidObj.read() 
        if not success:
            print("Video End")
            return 
        count += 1

    cv2.imwrite("images/frame4.jpg", image) 


    

In [37]:
FrameCapture("images/original_video.mp4")

## 1D Chaotic Map

- Logistic Map

In [22]:
def LogisticMap(dimension, key, k):
    
    x = key
    a = 3.6

    bitSequence = []
    matrix = []

    for i in range(dimension * k):
        x_next = a * x * (1 - x) 
        x = x_next
        if x <= 0.4:
            bit = 0
        else:
            bit = 1

        bitSequence.append(bit) 

        # If Bit Sequence has k bits, convert it to decimal and add it to Byte Sequence.
        if i % k == k - 1:
            decimal = binaryToDecimal(bitSequence)
            matrix.append(decimal)
            bitSequence = []

    return matrix

In [33]:
print(LogisticMap(6, 0.6, 4))

[14, 14, 14, 14, 14, 14]


## 2D Chaotic Map

- Hennon Map Implementation

In [24]:
def HennonMap(dimension, key, k):

    x = key[0]
    y = key[1]
    a = 1.4
    b = 0.3

    # Total Number of bitSequence produced
    sequenceSize = dimension * dimension * k 
    bitSequenceSize = k
    byteArraySize = dimension * k
    # Each bitSequence contains k bits
    bitSequence = []    
    # Each byteArray contains m bitSequence
    byteArray = []      
    # Each matrix contains m*n byteArray
    matrix = []

    for i in range(sequenceSize):
        x_next = y + 1 - (a * (x**2)) 
        y_next = b * x
        x = x_next
        y = y_next
        if x <= 0.4:
            bit = 0
        else:
            bit = 1

        bitSequence.append(bit) 

        # If Bit Sequence has k bits, convert it to decimal and add it to Byte Sequence.
        if i % bitSequenceSize == bitSequenceSize - 1:
            decimal = binaryToDecimal(bitSequence)
            byteArray.append(decimal)
            bitSequence = []
        
        # If Byte Sequence has k bytes, add it to Matrix.
        if i % byteArraySize == byteArraySize - 1:
            matrix.append(byteArray)
            byteArray = []

    return matrix

In [25]:
h = HennonMap(4, [0.1,0.1], 3)
print(h)

[[5, 2, 1, 6], [4, 2, 4, 7], [2, 1, 2, 3], [7, 7, 5, 2]]


- Hennon Map (Binary)

In [26]:
def HennonMapBinary(dimension, key):

    rows = dimension[0]
    cols = dimension[1]
    x = key[0]
    y = key[1]
    a = 1.4
    b = 0.3

    # Total Number of bitSequence produced
    sequenceSize = rows * cols
    bitSequenceSize = cols
    # Each bitSequence contains k bits
    bitSequence = []       
    # Each matrix contains m*n byteArray
    matrix = []

    for i in range(sequenceSize):
        x_next = y + 1 - (a * (x**2)) 
        y_next = b * x
        x = x_next
        y = y_next
        if x <= 0.4:
            bit = 0
        else:
            bit = 1

        bitSequence.append(bit) 

        # If Bit Sequence has k bits, convert it to decimal and add it to Byte Sequence.
        if i % bitSequenceSize == bitSequenceSize - 1:
            matrix.append(bitSequence)
            bitSequence = []

    return matrix

In [27]:
h = HennonMapBinary([3,4], [0.1,0.2])
print(h)

[[1, 0, 0, 1], [0, 1, 0, 1], [0, 1, 1, 1]]


## Encoding


In [40]:
def encode(filename, hennonKey, poly):

    # read the frames
    imagehandler1 = Image.open("images/frame1.jpg") 
    image1 = imagehandler1.load()

    imagehandler2 = Image.open("images/frame2.jpg") 
    image2 = imagehandler2.load()

    imagehandler3 = Image.open("images/frame3.jpg") 
    image3 = imagehandler3.load()

    imagehandler4 = Image.open("images/frame3.jpg") 
    image4 = imagehandler4.load()

    image_size = imagehandler1.size

    # Get secret data
    filehandler = open(filename,"r")
    secret_data = "".join(filehandler.readlines())

    # convert data to binary
    binary_secret_data = convertToBinary(secret_data)
    data_len = len(binary_secret_data)

    print(data_len)

    # Build chaotic map and derive some available pixels
    map = HennonMapBinary(image_size, hennonKey)

    availablePixels = []

    for i in range(image_size[0]):
        for j in range(0,image_size[1]):
            if map[i][j] == 1:
                availablePixels.append([i,j,0])

    # Build LFSR
    lfsr = LFSR(initstate = [1, 1, 1], fpoly=poly, counter_start_zero=True)

    index = 0

    for i in range(0, data_len, 8):

        # Choose an unused pixel using LFSR from Chaotic Map

        index = (index + binaryToDecimal(lfsr.state)) % len(availablePixels)
        lfsr.next()

        while availablePixels[index][2] >= 2:
            index = (index + binaryToDecimal(lfsr.state)) % len(availablePixels)
            print(index)
            lfsr.next()

        '''
        EACH PIXEL CAN BE USED AT MAX 3 TIMES. WE KEEP COUNT OF NUMBER OF USES AS USAGESTATE.

        IF USAGESTATE IS 0: XOR 2 DATA BITS WITH 2 LSB OF RED VALUE
        IF USAGESTATE IS 1: XOR 2 DATA BITS WITH 2 LSB OF BLUE VALUE
        IF USAGESTATE IS 2: XOR 2 DATA BITS WITH 2 LSB OF GREEN VALUE
        '''

        row, col, usageState = availablePixels[index][0], availablePixels[index][1], availablePixels[index][2]
        
        # Update with Red Value
        if usageState == 0:

            # Bits i and i+1
            value = list(convertToBinary(image1[row,col][0]))
            value[-2] = str(int(value[-2]) ^ int(binary_secret_data[i]))
            value[-1] = str(int(value[-1]) ^ int(binary_secret_data[i+1]))
            value = "".join(value)
            value = binaryToDecimal(value)
            image1[row,col] = (value, image1[row,col][1], image1[row,col][2])

            # Bits i+2 and i+3
            value = list(convertToBinary(image2[row,col][0]))
            value[-2] = str(int(value[-2]) ^ int(binary_secret_data[i+2]))
            value[-1] = str(int(value[-1]) ^ int(binary_secret_data[i+3]))
            value = "".join(value)
            value = binaryToDecimal(value)
            image2[row,col] = (value, image1[row,col][1], image2[row,col][2])

            # Bits i+4 and i+5
            value = list(convertToBinary(image3[row,col][0]))
            value[-2] = str(int(value[-2]) ^ int(binary_secret_data[i+4]))
            value[-1] = str(int(value[-1]) ^ int(binary_secret_data[i+5]))
            value = "".join(value)
            value = binaryToDecimal(value)
            image3[row,col] = (value, image3[row,col][1], image3[row,col][2])

            # Bits i+6 and i+7
            value = list(convertToBinary(image4[row,col][0]))
            value[-2] = str(int(value[-2]) ^ int(binary_secret_data[i+5]))
            value[-1] = str(int(value[-1]) ^ int(binary_secret_data[i+7]))
            value = "".join(value)
            value = binaryToDecimal(value)
            image4[row,col] = (value, image4[row,col][1], image4[row,col][2])

        # Update with Blue Value
        elif usageState == 1:

            # Bits i and i+1
            value = list(convertToBinary(image1[row,col][1]))
            value[-2] = str(int(value[-2]) ^ int(binary_secret_data[i]))
            value[-1] = str(int(value[-1]) ^ int(binary_secret_data[i+1]))
            value = "".join(value)
            value = binaryToDecimal(value)
            image1[row,col] = (image1[row,col][0], value, image1[row,col][2])

            # Bits i+2 and i+3
            value = list(convertToBinary(image2[row,col][1]))
            value[-2] = str(int(value[-2]) ^ int(binary_secret_data[i+2]))
            value[-1] = str(int(value[-1]) ^ int(binary_secret_data[i+3]))
            value = "".join(value)
            value = binaryToDecimal(value)
            image2[row,col] = (image2[row,col][0], value, image2[row,col][2])

            # Bits i+4 and i+5
            value = list(convertToBinary(image3[row,col][1]))
            value[-2] = str(int(value[-2]) ^ int(binary_secret_data[i+4]))
            value[-1] = str(int(value[-1]) ^ int(binary_secret_data[i+5]))
            value = "".join(value)
            value = binaryToDecimal(value)
            image3[row,col] = (image3[row,col][0], value, image3[row,col][2])

            # Bits i+6 and i+7
            value = list(convertToBinary(image4[row,col][0]))
            value[-2] = str(int(value[-2]) ^ int(binary_secret_data[i+5]))
            value[-1] = str(int(value[-1]) ^ int(binary_secret_data[i+7]))
            value = "".join(value)
            value = binaryToDecimal(value)
            image4[row,col] = (image4[row,col][0], value, image4[row,col][2])
        
        # Update with Green Value
        else:

            # Bits i and i+1
            value = list(convertToBinary(image1[row,col][1]))
            value[-2] = str(int(value[-2]) ^ int(binary_secret_data[i]))
            value[-1] = str(int(value[-1]) ^ int(binary_secret_data[i+1]))
            value = "".join(value)
            value = binaryToDecimal(value)
            image1[row,col] = (image1[row,col][0], value, image1[row,col][2])

            # Bits i+2 and i+3
            value = list(convertToBinary(image2[row,col][1]))
            value[-2] = str(int(value[-2]) ^ int(binary_secret_data[i+2]))
            value[-1] = str(int(value[-1]) ^ int(binary_secret_data[i+3]))
            value = "".join(value)
            value = binaryToDecimal(value)
            image2[row,col] = (image2[row,col][0], value, image2[row,col][2])

            # Bits i+4 and i+5
            value = list(convertToBinary(image3[row,col][1]))
            value[-2] = str(int(value[-2]) ^ int(binary_secret_data[i+4]))
            value[-1] = str(int(value[-1]) ^ int(binary_secret_data[i+5]))
            value = "".join(value)
            value = binaryToDecimal(value)
            image3[row,col] = (image3[row,col][0], value, image3[row,col][2])

            # Bits i+6 and i+7
            value = list(convertToBinary(image4[row,col][0]))
            value[-2] = str(int(value[-2]) ^ int(binary_secret_data[i+5]))
            value[-1] = str(int(value[-1]) ^ int(binary_secret_data[i+7]))
            value = "".join(value)
            value = binaryToDecimal(value)
            image4[row,col] = (image4[row,col][0], value, image4[row,col][2])
        
        availablePixels[index][2] += 1

    # Save the final frames.
    imagehandler1.save("images\encoded_frame1.png", "PNG")
    imagehandler2.save("images\encoded_frame2.png", "PNG")
    imagehandler3.save("images\encoded_frame3.png", "PNG")
    imagehandler4.save("images\encoded_frame4.png", "PNG")

In [41]:
encode("text\input.txt", [0.1, 0.1], [3,2])

88
